In [0]:
from __future__ import print_function

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from PIL import Image
from IPython.display import display 
import matplotlib.pyplot as plt
from torchvision import datasets
import torchvision.transforms as transforms
import torchvision.models as models
import torchsummary
import numpy as np
from collections import namedtuple
from torch.utils.data.sampler import SubsetRandomSampler
from torchsummary import summary
from torch.autograd import Variable

import copy
import time
import glob
import re
import cv2

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [0]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
batch_size = 32
random_seed = 10
num_epochs = 128
initial_lr = 1e-3
checkpoint_dir = "/content/gdrive/My Drive/Colab_Notebooks/best-artworks-of-all-time/data/" 

log_interval = 20
checkpoint_interval = 200

#running_option = "test" 
running_option = "training" 

In [0]:
class ConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride):
        super(ConvLayer, self).__init__()
        reflection_padding = kernel_size // 2
        self.reflection_pad = nn.ReflectionPad2d(reflection_padding)
        self.conv2d = nn.Conv2d(in_channels, out_channels, kernel_size, stride)

    def forward(self, x):
        out = self.reflection_pad(x)
        out = self.conv2d(out)
        return out

    
    
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = ConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in1 = nn.InstanceNorm2d(channels, affine=True)
        self.conv2 = ConvLayer(channels, channels, kernel_size=3, stride=1)
        self.in2 = nn.InstanceNorm2d(channels, affine=True)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.relu(self.in1(self.conv1(x)))
        out = self.in2(self.conv2(out))
        out = out + residual
        return out    
    


class UpsampleConvLayer(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride, upsample=None):
        super(UpsampleConvLayer, self).__init__()
        self.upsample = upsample
        reflection_padding = kernel_size // 2
        self.reflection_pad = nn.ReflectionPad2d(reflection_padding)
        self.conv2d = nn.Conv2d(in_channels, out_channels, kernel_size, stride)

    def forward(self, x):
        x_in = x
        if self.upsample:
            x_in = nn.functional.interpolate(x_in, mode='nearest', scale_factor=self.upsample)
        out = self.reflection_pad(x_in)
        out = self.conv2d(out)
        return out
    

    
class TransformerNet(nn.Module):
    def __init__(self):
        super(TransformerNet, self).__init__()
        # Initial convolution layers
        self.encoder = nn.Sequential()
        
        self.encoder.add_module('conv1', ConvLayer(3, 32, kernel_size=9, stride=1))
        self.encoder.add_module('in1', nn.InstanceNorm2d(32, affine=True))
        self.encoder.add_module('relu1', nn.ReLU())
        self.encoder.add_module('maxpool1', nn.MaxPool2d(2,2))
        
        self.encoder.add_module('conv2', ConvLayer(32, 64, kernel_size=3, stride=2))
        self.encoder.add_module('in2', nn.InstanceNorm2d(64, affine=True))
        self.encoder.add_module('relu2', nn.ReLU())
        self.encoder.add_module('maxpool2', nn.MaxPool2d(2,2))
        
        self.encoder.add_module('conv3', ConvLayer(64, 128, kernel_size=3, stride=2))
        self.encoder.add_module('in3', nn.InstanceNorm2d(128, affine=True))
        self.encoder.add_module('relu3', nn.ReLU())
        self.encoder.add_module('maxpool3', nn.MaxPool2d(2,2))

        # Residual layers
        self.residual = nn.Sequential()
        
        for i in range(5):
            self.residual.add_module('resblock_%d' %(i+1), ResidualBlock(128))
        
        # Upsampling Layers
        self.decoder = nn.Sequential()
        self.decoder.add_module('upsample1', nn.Upsample(scale_factor=2))
        self.decoder.add_module('deconv1', UpsampleConvLayer(128, 64, kernel_size=3, stride=1, upsample=2))
        self.decoder.add_module('in4', nn.InstanceNorm2d(64, affine=True))
        self.decoder.add_module('relu4', nn.ReLU())

        self.decoder.add_module('upsample2', nn.Upsample(scale_factor=2))
        self.decoder.add_module('deconv2', UpsampleConvLayer(64, 32, kernel_size=3, stride=1, upsample=2))
        self.decoder.add_module('in5', nn.InstanceNorm2d(32, affine=True))
        self.decoder.add_module('relu5', nn.ReLU())

        self.decoder.add_module('upsample3', nn.Upsample(scale_factor=2))
        self.decoder.add_module('deconv3', ConvLayer(32, 3, kernel_size=9, stride=1))

        #Classifier

    def forward(self, X):
        encoder_output = self.encoder(X)
        residual_output = self.residual(encoder_output)
        decoder_output = self.decoder(residual_output)
        return decoder_output

In [0]:
def normalize_batch(batch):
    # normalize using imagenet mean and std
    mean = batch.new_tensor([0.485, 0.456, 0.406]).view(-1, 1, 1)
    std = batch.new_tensor([0.229, 0.224, 0.225]).view(-1, 1, 1)
    batch = batch.div_(255.0)
    return (batch - mean) / std

In [0]:
np.random.seed(random_seed)
torch.manual_seed(random_seed)

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(256),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.mul(255))
])

style_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.mul(255))
])    

train_dataset = datasets.ImageFolder("/content/gdrive/My Drive/Colab_Notebooks/best-artworks-of-all-time/images", transform) #FIXME
test_dataset = datasets.ImageFolder("/content/gdrive/My Drive/Colab_Notebooks/best-artworks-of-all-time/images", transform) #FIXME

num_train = len(train_dataset)
indices = list(range(num_train))
split = int(np.floor(0.1* num_train))

np.random.seed(random_seed)
np.random.shuffle(indices)

train_idx, test_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler = train_sampler)
test_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, sampler = test_sampler)
print("train dataset num:", len(train_loader))
print("test dataset num:",len(test_loader) )

def im_convert(tensor):
    image = tensor.clone().detach().numpy()
    image = image.transpose(1, 2, 0)
    # denormalize
    image = image * np.array([0.5, 0.5, 0.5] + np.array([0.5, 0.5, 0.5]))
    image = image.clip(0, 255)
    return image

dataiter = iter(train_loader)
images, classes = dataiter.next()
title = [train_dataset.classes[i] for i in classes]

fig = plt.figure(figsize=(25, 4))

for i in np.arange(20):
    # row 2 column 10
    ax = fig.add_subplot(2, 10, i+1, xticks=[], yticks=[])
    plt.imshow(im_convert(images[i]).astype('uint8'))
    ax.set_title(title[i])

In [0]:
transformer = TransformerNet().to(device)
summary(transformer, (3, 256, 256))

In [0]:
optimizer = torch.optim.Adam(transformer.parameters(), initial_lr)
criterion = nn.MSELoss()

transfer_learning = True # inference or training first --> False / Transfer learning --> True
ckpt_model_path = os.path.join(checkpoint_dir, "ckpt_epoch_85_batch_id_200.pth") #FIXME

if transfer_learning:
    checkpoint = torch.load(ckpt_model_path, map_location=device)
    transformer.load_state_dict(checkpoint['model_state_dict'])
    transformer.to(device)

In [0]:
if running_option == 'training':
    if transfer_learning:
        transfer_learning_epoch = checkpoint['epoch']
    else:
        transfer_learning_epoch = 0
        
    for epoch in range(transfer_learning_epoch, num_epochs):
        transformer.train()
        agg_style_loss=0.
        count=0
        
        for batch_id, (x,labels) in enumerate(train_loader):
            n_batch = len(x)
            count += n_batch
            optimizer.zero_grad()
            
            # ================= forward =====================
            labels = labels.to(device)
            x = x.to(device)
            y = transformer(x)
            y = normalize_batch(y)
            x = normalize_batch(x)
            
            # ================= loss =====================
            loss = criterion(y, x)

            # =============== Backward ===================
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (batch_id + 1) % log_interval == 0:
                mesg = "{}\tEpoch {}:\t[{}/{}]\ttotal: {:.6f}".format(
                    time.ctime(), epoch + 1, count, len(train_dataset),
                                  loss.data
                )
                print(mesg)
            
            if checkpoint_dir is not None and (batch_id + 1) % checkpoint_interval == 0:
                transformer.eval().cpu()
                ckpt_model_filename = "ckpt_epoch_" + str(epoch) + "_batch_id_" + str(batch_id + 1) + ".pth"
                print(str(epoch), "th checkpoint is saved!")
                ckpt_model_path = os.path.join(checkpoint_dir, ckpt_model_filename)
                torch.save({
                'epoch': epoch,
                'model_state_dict': transformer.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': loss.data
                }, ckpt_model_path)

                transformer.to(device).train()

In [0]:
correct = 0
total = 0
if running_option == "test":
  with torch.no_grad():
        style_model = TransformerNet()
        
        ckpt_model_path = os.path.join(checkpoint_dir, "ckpt_epoch_10_batch_id_200.pth") #FIXME
        checkpoint = torch.load(ckpt_model_path, map_location=device)

        style_model.load_state_dict(checkpoint['model_state_dict'])
        style_model.to(device)    
        for x, labels in test_loader :
            x = x.to(device)
            labels = labels.to(device)
            y, outputs = style_model(x)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))


https://excelsior-cjh.tistory.com/187 - sparse loss function
